In [ ]:
# It's python lib for our boosting model! Install it for the FIRST time only. https://catboost.ai/en/docs/installation/python-installation-method-pip-install
#!pip install catboost 

In [60]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

import catboost
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


In [61]:
print(np.__version__)
print(pd.__version__)
print(catboost.__version__)
!python --version

1.26.4
2.2.1
1.2.3
Python 3.12.2


In [62]:
data = pd.read_csv('../raw_data/merged_out.csv')
data.head(2)

,Unnamed: 0,pipeline-1__R_KD,pipeline-1__B_KD,pipeline-1__R_TD_pct,pipeline-1__B_TD_pct,pipeline-1__R_SUB_ATT,pipeline-1__B_SUB_ATT,pipeline-1__R_REV,pipeline-1__B_REV,pipeline-1__R_CTRL,...,pipeline-2__R_fighter,pipeline-2__B_fighter,pipeline-2__win_by,pipeline-2__Format,pipeline-2__Fight_type,pipeline-2__R_fighter_name,pipeline-2__R_Stance,pipeline-2__B_fighter_name,pipeline-2__B_Stance,pipeline-2__R_Win
0,0,2.0,0.0,0.0,-1.255385,0.0,0.0,0.0,0.0,-0.351111,...,Adrian Yanez,Gustavo Lopez,KO/TKO,3 Rnd (5-5-5),Bantamweight Bout,Adrian Yanez,Orthodox,Gustavo Lopez,Orthodox,True
1,1,0.0,0.0,0.0,0.104615,1.0,2.0,0.0,1.0,-0.031111,...,Trevin Giles,Roman Dolidze,Decision - Unanimous,3 Rnd (5-5-5),Middleweight Bout,Trevin Giles,Orthodox,Roman Dolidze,Orthodox,True


In [63]:
X = data.drop(['pipeline-2__R_Win'], axis=1)
y= data['pipeline-2__R_Win']

In [64]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6012 entries, 0 to 6011
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    6012 non-null   int64  
 1   pipeline-1__R_KD              6012 non-null   float64
 2   pipeline-1__B_KD              6012 non-null   float64
 3   pipeline-1__R_TD_pct          6012 non-null   float64
 4   pipeline-1__B_TD_pct          6012 non-null   float64
 5   pipeline-1__R_SUB_ATT         6012 non-null   float64
 6   pipeline-1__B_SUB_ATT         6012 non-null   float64
 7   pipeline-1__R_REV             6012 non-null   float64
 8   pipeline-1__B_REV             6012 non-null   float64
 9   pipeline-1__R_CTRL            6012 non-null   float64
 10  pipeline-1__B_CTRL            6012 non-null   float64
 11  pipeline-1__last_round        6012 non-null   float64
 12  pipeline-1__last_round_time   6012 non-null   float64
 13  pip

In [65]:
col_names = X.columns.to_list()
categorical_column_names = X.select_dtypes(include=['object']).columns.to_list()
categorical_indices = [col_names.index(i) for i in categorical_column_names]
categorical_indices

[53, 54, 55, 56, 57, 58, 59, 60, 61]

In [66]:
categorical_column_names

['pipeline-2__R_fighter',
 'pipeline-2__B_fighter',
 'pipeline-2__win_by',
 'pipeline-2__Format',
 'pipeline-2__Fight_type',
 'pipeline-2__R_fighter_name',
 'pipeline-2__R_Stance',
 'pipeline-2__B_fighter_name',
 'pipeline-2__B_Stance']

In [67]:
y.value_counts(normalize=True)

pipeline-2__R_Win
True     0.661843
False    0.338157
Name: proportion, dtype: float64

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, 
                                                       train_size=0.8, 
                                                       random_state=42, stratify = y)


In [69]:
X_train = pd.DataFrame(data=X_train, columns=X.columns)
X_test = pd.DataFrame(data=X_test, columns=X.columns)

In [70]:
cv = StratifiedKFold(n_splits = 5)
merged_model1 = catboost.CatBoostClassifier(n_estimators=2000, depth=5, learning_rate=0.038,silent=True,
                                             cat_features=categorical_indices,
                                             eval_metric='AUC')

In [71]:
merged_pipe1 = Pipeline([
    ("merged_classifier", merged_model1)
])
    
merged_pipe1

Pipeline(steps=[('merged_classifier',
                 <catboost.core.CatBoostClassifier object at 0x7f2289c7acc0>)])

In [72]:
merged_pipe1_mean_accuracy = cross_val_score(merged_pipe1, X_train, y=y_train, scoring='accuracy', cv=cv).mean()
merged_pipe1_mean_accuracy

0.8997732784413325

In [73]:
merged_pipe1.fit(X_train,y_train)
y_pred = merged_pipe1.predict(X_test)

In [74]:
merged_pipe1_accuracy = accuracy_score(y_test, y_pred)
merged_pipe1_accuracy

0.8919368246051538

In [75]:
#Export the fitted pipeline as a pickle file
with open('../models/merged_model1_acc08998.pkl', 'wb') as file:
    pickle.dump(merged_pipe1, file)
print("merged_pipe1 is successfully saved as 'merged_pipe1_acc08998.pkl'")

merged_pipe1 is successfully saved as 'merged_pipe1_acc08998.pkl'
